In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-ecommerce-data/2019-Dec-Cleaned.csv
/kaggle/input/cleaned-ecommerce-data/2019-Nov-Cleaned.csv


In [2]:
# File path for the December dataset
december_file = "/kaggle/input/cleaned-ecommerce-data/2019-Dec-Cleaned.csv"

# Load the December dataset
december_df = pd.read_csv(december_file)

# Display December data preview
print("December Data Preview:")
print(december_df.head())

December Data Preview:
                event_time event_type  product_id          category_id  \
0  2019-12-01 00:00:00 UTC       view     1005105  2232732093077520756   
1  2019-12-01 00:00:01 UTC       view     2402273  2232732100769874463   
2  2019-12-01 00:00:02 UTC       view    20100164  2232732110089618156   
3  2019-12-01 00:00:02 UTC       view   100008256  2053013561185141473   
4  2019-12-01 00:00:03 UTC       view     1005239  2232732093077520756   

                  category_code   brand    price    user_id  \
0      construction.tools.light   apple  1302.48  556695836   
1  appliances.personal.massager   bosch   313.52  539453785   
2              apparel.trousers    nika   101.68  517987650   
3          accessories.umbrella    ikea   163.56  542860793   
4      construction.tools.light  xiaomi   256.38  525740700   

                           user_session  
0  ca5eefc5-11f9-450c-91ed-380285a0bc80  
1  5ee185a7-0689-4a33-923d-ba0130929a76  
2  906c6ca8-ff5c-419a-bde9-

In [3]:
# Feature 1: Total Spent
december_df["total_spent"] = december_df["price"]

# Feature 2: Extract Date-Time Features
december_df["event_time"] = pd.to_datetime(december_df["event_time"])
december_df["hour"] = december_df["event_time"].dt.hour
december_df["day"] = december_df["event_time"].dt.day
december_df["month"] = december_df["event_time"].dt.month

# Feature 3: Aggregate User Data
user_agg_dec = december_df.groupby("user_id").agg(
    avg_spent_per_user=("total_spent", "mean"),
    num_sessions=("user_session", "nunique")  # Count unique sessions per user
).reset_index()

# Save the processed features
user_agg_dec.to_csv("/kaggle/working/december_processed_data.csv", index=False)


In [4]:
# XGBoost Section
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Add dummy 'target' column for demonstration (replace with actual target column if available)
import numpy as np
user_agg_dec["target"] = np.random.choice([0, 1], size=len(user_agg_dec))

# Split features (X) and target (y)
X = user_agg_dec.drop(["user_id", "target"], axis=1)
y = user_agg_dec["target"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = XGBClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.49910312213845787
